#یادگیری خود نظارتی با تخمین چرخش

## در اینجا میخاییم  یک مدل رو پریترین کنیم ینی با استفاده از مسئله کمکی مدل رو برای مسئله اصلی اماده و سپس تنظیم دقیق کنیم

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import random
from PIL import Image

In [2]:
# Define the RotNet model
class RotNet(nn.Module):
    def __init__(self):
        super(RotNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 8 * 8, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 4)  # 4 possible rotations: 0, 90, 180, 270
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [3]:
# Generate rotated images
def generate_rotated_images(dataset):
    rotated_images, labels = [], []
    for img, _ in dataset:
        for angle, label in zip([0, 90, 180, 270], [0, 1, 2, 3]):
            img_pil = transforms.ToPILImage()(img)
            img_rot = transforms.functional.rotate(img_pil, angle)
            img_tensor = transforms.ToTensor()(img_rot)
            rotated_images.append(img_tensor)
            labels.append(label)
    return rotated_images, labels

In [4]:
# Load and preprocess data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:02<00:00, 68.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# Self-supervised learning on the unlabeled dataset
rotnet = RotNet()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
rotnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rotnet.parameters(), lr=0.001)

rotated_images, labels = generate_rotated_images(trainset)
rotated_dataset = list(zip(rotated_images, labels))
rotated_loader = DataLoader(rotated_dataset, batch_size=64, shuffle=True)

In [7]:
labels[:10]

tensor([0, 2, 1, 2, 1, 2, 3, 3, 2, 0])

شبه برچسب ها هستند که از 1 تا 4 نشانگر چرحش های 90 180 270 360 میباشد

In [ ]:
# Training RotNet
for epoch in range(10):
    rotnet.train()
    running_loss = 0.0
    for images, labels in rotated_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = rotnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(rotated_loader)}')

# Save the model weights for fine-tuning
torch.save(rotnet.state_dict(), 'rotnet_weights.pth')

In [8]:
# Define the Fine-Tuned model
class FineTunedNet(nn.Module):
    def __init__(self, rotnet):
        super(FineTunedNet, self).__init__()
        self.features = rotnet.features
        self.classifier = nn.Sequential(
            nn.Linear(128 * 8 * 8, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 10)  # 10 classes for CIFAR-10
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [9]:
# Fine-tuning with a small percentage of the labeled dataset
def fine_tune_model(rotnet, labeled_dataset, labeled_percentage):
    fine_tuned_model = FineTunedNet(rotnet)
    fine_tuned_model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(fine_tuned_model.parameters(), lr=0.001)

    # Shuffle and take a small percentage of the labeled data
    labeled_size = int(len(labeled_dataset) * labeled_percentage)
    indices = random.sample(range(len(labeled_dataset)), labeled_size)
    subset = Subset(labeled_dataset, indices)
    subset_loader = DataLoader(subset, batch_size=64, shuffle=True)

    # Training Fine-Tuned Model
    for epoch in range(10):
        fine_tuned_model.train()
        running_loss = 0.0
        for images, labels in subset_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = fine_tuned_model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Fine-tuning Epoch {epoch + 1}, Loss: {running_loss / len(subset_loader)}')

    return fine_tuned_model

In [10]:
# Evaluate the model
def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [ ]:
# Example usage: fine-tuning with 1%, 5%, and 10% of the labeled dataset
labeled_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_loader = DataLoader(testset, batch_size=64, shuffle=False)

percentages = [0.01, 0.05, 0.1]
for perc in percentages:
    print(f"Fine-tuning with {int(perc * 100)}% of the labeled dataset.")
    fine_tuned_model = fine_tune_model(rotnet, labeled_dataset, perc)
    accuracy = evaluate_model(fine_tuned_model, test_loader, criterion, device)
    print(f"Test Accuracy with {int(perc * 100)}% labeled data: {accuracy:.2f}%")